# tRAT 3

---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London 

**Module:** ELEC70066 - Advanced Optimisation

**Version:** 1.1.0 - 12/01/2023

---
---

GROUP 7

# Exercise

Consider a three-state boolean linear programme

$$
\begin{array}{ll}
\displaystyle \min_{x} &  c^\top x \\ 
\text{s.t. } & Ax \preccurlyeq b,\\
& x_i\in \{0, 0.5, 1\}, \qquad i = 1, \dots, n.
\end{array}\tag{1}
$$

where the variable $x$ is constrained to have components equal to zero, half or one. You can think of $x_i$ as a job we can accept (1), subcontract (0.5) or decline (0), and $−c_i$ as the (positive) revenue we generate if we accept or subcontract job $i$. We can think of $Ax \preccurlyeq b$ as a set of limits on $m$ resources. $A_{ij}$, which is positive, is the amount of resource $i$ consumed if we accept job $j$; If the job is subcontracted we have half the revenues, but we also half the amount of resources spent. $b_i$, which is positive, is the amount of resource $i$ available.

For your convenience, the relaxation of the problem (i.e. $0 \le x\le 1$) is already solved in the code below.

In [ ]:
import cvxpy as cp
import numpy as np

n = 100
m = 300
np.random.seed(1)
A = np.random.random((m, n)) # positve
b = np.dot(A,np.ones(n)/2) # postive
c = -np.random.random(n) # negative, so -c is positive

In [ ]:
# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(c.T@x),[A @ x <= b, x >= 0, x<= 1])
prob.solve()

# Print result.
L=prob.value
print("\nThe optimal value is", prob.value)
#print("A solution x is", x.value)


The optimal value is -34.589711316657315


1.   Explain how you would develop a threshold rounding for this problem.
2.   Carry out threshold rounding. For each value of of the threshold(s), note the objective value $c^\top \hat{x}$ and the maximum constraint violation $\max_i(A\hat{x} − b)_i$. 
3.   Find the minimum feasible objective value and note it as the upper bound $U$. Compute the relative suboptimality percentage as $\frac{|U-L|}{|L|} \times 100$.

1- I would create a meshgrid of t1, and t2, where both t1 and t2 range between 0 and 1 and take 100 unique values each, hence the meshgrid has 10000 element.s
Round x <t1 to 0, t1<=x<=t2 to 0.5, and t2<x to 1
I then calculate the cost and constraint vialation for each, and pick the threshold that with the lowest cost that satisfies the constraint
- Below I used for loops for t1 and t2 rather than creating a meshgrid, but its practically the same

In [ ]:
X = x.value
cost_array = np.zeros((n,n))
cost_array_non_violated = np.zeros((n,n))
cost_array_violated = np.zeros((n,n))
const_violation = np.zeros((n,n))

for i in range(n):
  t1 = i/n
  for k in range(n):
    t2 = k/n

    x_temp = X.copy()

    arr1 = np.where(X<=t1)
    arr2 = np.where((X>t1) & (X<t2))
    arr3 = np.where(X>=t2)
    
    x_temp[arr1] = 0
    x_temp[arr2] = 0.5
    x_temp[arr3] = 1
    
    const_violation[i][k] = np.max(np.matmul(A,x_temp) -b)
    cost_array[i][k] = np.matmul(c.T,x_temp)

    if(np.max(np.matmul(A,x_temp) -b) > 0):
      cost_array_violated[i][k] = np.matmul(c.T,x_temp)
    else:
        cost_array_non_violated[i][k] = np.matmul(c.T,x_temp)
      


In [ ]:
min_valid_cost = cost_array_non_violated.min()
print("Minimum valid cost: ",min_valid_cost)

Minimum valid cost:  -33.94322461620416


In [ ]:
index = np.where(cost_array_non_violated == cost_array_non_violated.min())
index
# Hence multiple points with the same value satisfy the minimum

(array([36, 36, 36, 36, 36, 36, 36]), array([64, 65, 66, 67, 68, 69, 70]))

In [ ]:
const_violation[index]

array([-0.12559138, -0.12559138, -0.12559138, -0.12559138, -0.12559138,
       -0.12559138, -0.12559138])

In [ ]:
# they also have the same constraint violation value as well

so the optimal points are t1 = 0.36, t2=[0.64-0.70]

In [ ]:
sub_opt = np.abs(min_valid_cost - L) / np.abs(L)*100
print("Relative suboptimality percentage is: ",sub_opt)

Relative suboptimality percentage is:  1.8690144434418137
